In [3]:
from datasets import load_dataset
import random

In [ ]:
dataset = load_dataset("openai/gsm8k", "main")
print (dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})


In [5]:
train_random_idx = random.randint(0, len(dataset['train']) - 1)
test_random_idx = random.randint(0, len(dataset['test']) - 1)

print ("\nTraining sample:\n")
print (f"Question ===>\t{dataset['train'][train_random_idx]['question']}")
print (f"Answer ===>\t{dataset['train'][train_random_idx]['answer']}")

print ("\nTesting sample:\n")
print (f"Question ===>\t{dataset['test'][test_random_idx]['question']}")
print (f"Answer ===>\t{dataset['test'][test_random_idx]['answer']}")


Training sample:

Question ===>	A pet store has six dogs for sale. They have half as many cats, twice as many birds, and thrice as many fish for sale. How many animals are for sale in the pet store?
Answer ===>	The pet store has 6/2 = <<6/2=3>>3 cats for sale.
They have 6 x 2 = <<6*2=12>>12 birds for sale.
They have 6 x 3 = <<6*3=18>>18 birds for sale.
Therefore, there are a total of 6 + 3 + 12 + 18 = <<6+3+12+18=39>>39 pets for sale.
#### 39

Testing sample:

Question ===>	Stephen placed an online order for groceries.  His final bill came to $40.00.  Because this was through a delivery vendor, they tacked on a 25% fee to his final total and charged him $3.00 in delivery fees.  Stephen also added a $4.00 tip.  After the extra fees, what was the final price of Stephen's groceries?
Answer ===>	He spent $40.00 on groceries but they charged him a 25% fee so that's 40*.25 = $10.00
There is also a $3.00 delivery fee and a $4.00 tip for an extra $3 + $4 = $<<3+4=7.00>>7.00 in fees
His grocer

In [6]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

# quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = Gemma3ForCausalLM.from_pretrained(
    model_id
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [8]:
model.__class__.__name__

'Gemma3ForCausalLM'

In [11]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")

model_size = sum(p.element_size() * p.numel() for p in model.parameters()) / 1024 ** 2
print (f"Model size: {model_size:.2f} MB")

Total parameters: 999,885,952
Model size: 3814.26 MB


In [12]:
for name, layer in model.named_children():
    print (f"{name}: {layer}")

model: Gemma3TextModel(
  (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
  (layers): ModuleList(
    (0-25): 26 x Gemma3DecoderLayer(
      (self_attn): Gemma3Attention(
        (q_proj): Linear(in_features=1152, out_features=1024, bias=False)
        (k_proj): Linear(in_features=1152, out_features=256, bias=False)
        (v_proj): Linear(in_features=1152, out_features=256, bias=False)
        (o_proj): Linear(in_features=1024, out_features=1152, bias=False)
        (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
        (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
      )
      (mlp): Gemma3MLP(
        (gate_proj): Linear(in_features=1152, out_features=6912, bias=False)
        (up_proj): Linear(in_features=1152, out_features=6912, bias=False)
        (down_proj): Linear(in_features=6912, out_features=1152, bias=False)
        (act_fn): PytorchGELUTanh()
      )
      (input_layernorm): Gemma3RMSNorm((1152,), eps=1e-06)
      (post_attention_layernorm): Gemma3RM

In [7]:
messages = [
    [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant."},]
        },
        {
            "role": "user",
            "content": [{"type": "text", "text": "Write a rhyming poem about the heatwave in Rajasthan."},]
        },
    ],
]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device).to(torch.bfloat16)


with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=1000)

outputs = tokenizer.batch_decode(outputs)


Attempting to cast a BatchEncoding to type torch.bfloat16. This is not supported.


KeyboardInterrupt: 

In [ ]:
# TODO: How to make the inference faster? And when the inference is fast, I can load larger models


In [ ]:
print (outputs[0].split("<start_of_turn>model")[1])